# Create/Connect to Dask Cluster

In this example we just launch one locally, but you can just as well connect to existing one.

## Cluster sizing

Majority of the "work" is waiting for S3 data to arrive, so you would want to oversubscribe your cluster, i.e. have way more workers than there are CPUs, 8 workers per core is not unreasonable. 

Should you use threads or processes?

I recommend more threads, many threads per worker process allows sharing of common data more efficiently, the downside is [GIL](https://wiki.python.org/moin/GlobalInterpreterLock), so having too many threads might become problematic. Most of the time is spent waiting for HTTP data (from S3), GIL is released during this time. Ultimately one has to experiment to see what works best for your workload. Important message is: "don't be afraid to use more threads."

## Use local worker pool when still debugging

In the code below we launch local cluster in the same process that runs this notebook. This makes debugging any problems easier.

In [ ]:
import dask
import dask.distributed

client = dask.distributed.Client(n_workers=1, 
                                 threads_per_worker=32, 
                                 processes=False, 
                                 ip='127.0.0.1')
client

# Configure Dask Cluster for S3 I/O

1. Configure GDAL for cloud access on every worker process
2. Check that we can obtain AWS credentials

## Note on STS

If using [STS](https://docs.aws.amazon.com/STS/latest/APIReference/Welcome.html) to obtain S3 access credentials, you have to keep in mind the following:

- Every worker thread will obtain its own set of credentials (first time it does IO)
- Token expiry will cause I/O errors
- To force credential renewal you have to call `set_default_rio_config` again on every worker

Most robust and efficient way is to create a locked down set of credentials that can only read s3 buckets of interest and provision that to every worker (`~/.aws/{config|credentials}`)

In [ ]:
def worker_setup_auto():
    from datacube.utils.rio import set_default_rio_config, activate_from_config
    
    # these settings will be applied in every worker thread
    set_default_rio_config(aws={'region_name': 'auto'},
                           cloud_defaults=True)
    
    # Force activation in the main thread
    # - Really just to test that configuration works
    # - Every worker thread will automatically run this again
    return activate_from_config()

# Runs once on every worker process, not per worker thread!
client.register_worker_callbacks(setup=worker_setup_auto)

In [ ]:
from IPython.display import display, Image
from types import SimpleNamespace
from datacube import Datacube
from odc.ui import (show_datasets,
                    to_rgba, 
                    to_jpeg_data)

dc = Datacube(env='gm')

cfg = SimpleNamespace(product='ls8_nbart_geomedian_annual',
                      time='2017',
                      crs='EPSG:3577',
                      resolution=(-32*25, 32*25),  # 1/32 of native res
                      dask_chunk=512,
                      measurements=('red', 'green', 'blue'))

## Find datasets

You can supply query directly to `dc.load`, but here we are doing it in 2 steps.

1. Find datasets matching query
2. Pass datasets to `dc.load`

The advantage of 2 step approach is that you can review the result of a query before committing to loading all that data.

In [ ]:
%%time
dss = dc.find_datasets(product=cfg.product, time=cfg.time)
print('Found {:,} datasets'.format(len(dss)))

## Visualise Query Result

In [ ]:
show_datasets(dss, 
              style={'fill': False, 
                     'weight': 1, 
                     'opacity': 0.3, 
                     'color': 'green'},
              scroll_wheel_zoom=True, 
              width='400px', 
              height='400px')

## Lazy Dask Array

Construct lazy dask array. Below cell should execute relatively quickly as it does not load pixel data, just creates a recipe for fetching data from S3.

Note that we are using `datasets=dss` instead of query parameters, which in our case is `time='2017'`. If you are not interested in previewing query result, you can just call `dc.load` directly with your query. Note also that you still have to pass in `product=` argument, it's a limitation of `dc.load`, it doesn't know that it can be extracted from `datasets=` parameter.

In [ ]:
%%time
xx = dc.load(product=cfg.product,
             datasets=dss,
             output_crs=cfg.crs,
             resolution=cfg.resolution,
             measurements=cfg.measurements,
             dask_chunks={'x': cfg.dask_chunk, 'y': cfg.dask_chunk})

print("Number of chunks per band: {}x{}x{}".format(*xx.red.data.to_delayed().shape))
display(xx)

## Actually load data

Note that `to_rgba` is non-dask aware function, yet it works just the same with dask arrays. It creates a `u8` RGBA image by scaling `red`/`green`/`blue` bands such that `0u16->0u8` and `clamp->255u8`, if `clamp` is not supplied maximum value across three bands is used, alpha channel is computed from `red` channel using `nodata` value.

### Gotchas

When calling out to non-dask aware code it is very easy to hit performance issues. Common problems are 

- Running out of memory
- Loading same data more than once
- Never sure when code will trigger data loading and when will just return another dask array without performing actual computation

In the specific case of `to_rgba`, code does equivalent of this early on:

```python
r,g,b = (x.values for x in (xx.red, xx.green, xx.blue))
```

This line is almost zero cost when `xx` is fully loaded dataset, but in the case of dask arrays this triggers data loading for `red`, `green` and `blue` channels in that order. You should be able to observe that on the dask dashboard. So this particular code won't trigger double pass over the data, but on the other hand it is more likely to trigger out of memory errors.

In [ ]:
%%time
cc = to_rgba(xx, clamp=3000)

In [ ]:
Image(data=to_jpeg_data(cc.isel(time=0).values, quality=30))

------------------------------------